본 파일은 이기창님의 'Do it! 자연어 처리'에 기초하여 작성되었음을 미리 알려드립니다! :)

# 문서분류 모델 학습하기




## 1. 각종 설정하기

### TPU 관련 패키지 설치

코랩 노트북 초기화 과정에서 하드웨어 가속기로 TPU를 선택했다면 다음 코드를 실행하면 된다.
그러면 TPU 관련 라이브러리 들을 설치한다.

(참고로 TPU 학습은 라이브러리 지원 등이 GPU보다 불안정한 편이므로 될 수 있으면 GPU를 사용하기를 권함)

#### code 1-0

In [1]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

  Using cached https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl (149.9 MB)


### 의존성 패키지 설치

다음 코드는 TPU이외에 의존성 있는 패키지를 설치한다. 

명령어 맨 앞에 붙은 느낌표(!)는 코랩 환경에서 파이썬이 아닌, 셸(shell)명령을 수행한다는 의미이다.

#### code 1-1

In [1]:
!pip install ratsnlp

### 구글 드라이브와 연결

코랩 노트북은 일정시간 사용하지 않으면 당시까지의 모든 결과물이 날아갈 수 있다. 모델 체크포인트 등을 저장해 두기 위해 자신의 구글 드라이브를 코랩 노트북과 연결한다.

#### code 1-2

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


### 모델 환경 설정
kcbert-base 모델을 NSMC데이터로 파인튜닝

#### code 1-3

In [3]:
import torch
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="nsmc",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-doccls",
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=128,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

참고로 TrainArguments의 각 인자가 하는 역할과 의미는 다음과 같다.



*    **`pretrained_model_name`**
> 프리트레인 마친 언어 모델의 이름 (단, 해당 모델은 허깅페이스 모델 허브에 등록되어 있어야 한다.)

*    **`downstream_corpus_name`**
> 다운스트림 데이터의 이름

*    **`downstream_corpus_root_dir`**
> 다운스트림 데이터를 내려받을 위치. 입력하지 않으면 /root/Korpora에 저장된다.

*    **`downstream_model_dir`**
> 파인튜닝된 모델의 체크포인트가 저장될 위치. gdrive/My Drive/nlpbook/checkpoint-doccs로 지정하면 자신의 구글 드라이브 [내 폴더] 아래 npbook/checkpoint-doccls디렉터리에 저장된다.

*    **`batch_size`**
> 배치 크기. 하드웨어 가속기로 GPU를 선택(`torch.cuda.is_available() == True`)했다면 32, TPU라면(`torch.cuda.is_available() == False`) 4. 코랩 환경에서 TPU는 보통 8개의 코어가 할당되는데 batch_size는 코어별로 적용되는 배치 크기이므로 이렇게 설정해 둔다.

*    **`learning_rate`**
> 러닝 레이트(보폭). 1회 스텝에서 모델을 얼마나 업데이트할지에 관한 크기를 가리킨다. 

*   **`max_seq_length`**
> 토큰 기준 입력 문장 최대 길이. 이보다 긴 문장은 `max_seq_length`로 자르고, 짧은 문장은 `max_seq_length`가 되도록 스페셜 토큰([PAD])를 붙여준다.

*    **`epochs`**
> 학습 에포크 수. 3이라면 학습 데이터 전체를 3회 반복 학습합니다.

*    **`tpu_cores`**
> TPU 코어 수. 하드웨어 가속기로 GPU를 선택했다면 0, TPU라면 8.

*    **`seed`**
> 랜덤 시드(정수). `None`을 입력하면 랜덤 시드를 고정하지 않는다.



### 랜덤 시드 고정

랜덤 시드를 설정.

code 1-4는 `args`에 지정된 시드로 고정하는 역할을 한다.

#### code 1-4

In [4]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


각종 로그를 출력하는 로거를 설정.

#### code 1-5

In [5]:
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='document-classification', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-doccls', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)


## 2. 말뭉치 내려받기

### 말뭉치 내려 받기

NSMC 데이터를 내려받는다. 데이터를 내려받는 도구로 **코포라**(Korpora*)라는 파이썬 오픈소스 패키지를 사용해, corpus_name(nsmc)에 해당하는 말뭉치를 root_dir(/root/Korpora) 아래에 저장해 둔다.

* Korpora*: github.com/ko-nlp/korpora

#### code 1-6

In [6]:
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=True,
) 

[nsmc] download ratings_train.txt: 14.6MB [00:00, 94.7MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 40.6MB/s]                            


## 3. 토크나이저 준비하기

### 토크나이저 준비

  본 프로젝트에서 다루는 데이터의 기본 단위는 텍스트 형태의 문장이다. 토큰화란 문장을 토큰 시퀀스로 분절하는 과정을 가리킨다. 본 실습에서 사용하는 모델은 자연어 문장을 분절한 토큰 시퀀스를 입력 받는다. 

code 1-7를 실행해 kcbert-base 모델이 사용하는 토크나이저를 선언한다.

토크나이저는 토큰화를 수행하는 프로그램이라는 뜻이다.

#### code 1-7

In [7]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

## 4. 데이터 전처리하기

딥러닝 모델을 학습하려면 학습 데이터를 배치 단위로 계속 모델에 공급해 주어야 한다. 파이토치에서는 이 역할을 **데이터 로더**(dataloader)가 수행한다.

데이터 로더는 **데이터셋**(dataset)이 보유하고 있는 인스턴스를 배치 크기만큼 뽑아서 자료형, 데이터 길이 등 정해진 형식에 맞춰 배치를 만들어 준다.

### 학습 데이터셋 구축

code 1-8을 통해 **ClassificationDataset**을 만들 수 있다.
**ClassificationDataset**의 가장 큰 역할은 모든 인스턴스를 가지고 있다가 데이터 로더가 배치를 만들 때 인스턴스를 제공하는 일이다.

#### code 1-8

In [8]:
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset
corpus = NsmcCorpus()
train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/nsmc/ratings_train.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 35.781 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 아 더빙.. 진짜 짜증나네요 목소리
INFO:ratsnlp:tokens: [CLS] 아 더 ##빙 . . 진짜 짜증나네 ##요 목소리 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

### **ClassificationDataset** 클래스가 하는 역할

이 클래스는 **NsmcCorpus**와 위에서 선언해 둔 **토크나이저(tokenizer)**를 품고 있다.

**NsmcCorpus**는 CSV 파일 형식의 NSMC 데이터를 문장과 레이블*(예를 들면 영화 리뷰와 긍정 및 부정)*으로 읽는다.

**NsmcCorpus**는 **ClassificationDataset**이 요구하면 이 문장과 레이블을 **ClassificationDataset**에 제공한다.

 **ClassificationDataset**은 제공받은 문장과 레이블 각각을 tokenizer를 활용해 모델이 학습할 수 있는 형태(**ClassificationFeature**)로 가공한다.

 **ClassificationFeature**라는 자료형에는 총 4가지의 정보가 있다.
* 첫번째는 **`input_id`**이다. 인덱스로 변환된 토큰 시퀀스이다. 

* 두번째는 **`attention_mask`**로 해당 토큰이 패딩 토큰인지(0) 아닌지(1)를 나타낸다. 

* **`token_type_ids`** 세그먼트 정보, **`label`**은 정수로 바뀐 레이블 정보 이다.

**`ClassificationFeatures`** 각 구성 요소의 자료형은 다음과 같다.

*   **`input_ids`** : `List[int]`
*   **`attention_mask`** : `List[int]`
*   **`token_type_ids`** : `Listh[int]`
*   **`label`** : `int`







### 학습 데이터 로더 구축

code 1-9를 통해 학습할 때 쓰이는 데이터 로더를 만들 수 있다. 데이터 로더는 ClassificationDataset클래스가 들고 있는 전체 인스턴스 가운데 배치 크기(*code 1-3에서 정의한* `args`*의* `batch_size`)만큼을 뽑아 배치 형태로 가공(`nlpbook.data_collator`)하는 역할을 수행한다.

#### code 1-9

In [9]:
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

코드를 자세히 보면 `sampler`와 `collate_fn`이 눈에 띈다.
전자는 샘플링 방식을 정의한다.

 여기서 만든 데이터 로더는 배치를 만들 때 `ClassificationDataset`이 들고 있는 전체 인스턴스 가운데 `batch_size` 개수만큼 비복원(**`replacement=False`**) 랜덤 추출(RandomSampler)한다.

`collate_fn`은 이렇게 뽑은 인스턴스들을 배치로 만드는 역할을 하는 함수이다. `nlpbook.data_collator`는 같은 배치에서 인스턴스가 여럿일 때 이를 `input_ids`, `attention_mask` 등 종류별로 모으고 파이토치가 요구하는 자료형인 텐서(`tensor`)형태로 바꾸는 역할을 수행한다.

### 평가용 데이터 로더 구축

평가용 데이터 로더는 학습용 데이터 로더와 달리 **`SequentialSampler`**를 사용한다. `SequentialSampler`는 인스턴스를 `batch_size`만큼 순서대로 추출하는 역할을 한다. 학습 때 배치 구성은 랜덤으로 하는 것이 좋은데, 평가할 때는 평가용 데이터 전체를 사용하므로 굳이 랜덤으로 구성할 이유가 없어 `SequentialSampler`를 사용한다.

code 1-10을 통해 평가용 데이터 로더를 구축한다.

#### code 1-10

In [10]:
from torch.utils.data import SequentialSampler
val_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)
val_dataloader= DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading test data... LOOKING AT /content/Korpora/nsmc/ratings_test.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 12.095 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 굳 ㅋ
INFO:ratsnlp:tokens: [CLS] 굳 ㅋ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

## 5. 모델 불러오기

### 모델 초기화

code 1-3**(1. 각종 설정하기-모델 환경 설정)**에서 `pretrained_model_name`을 `beomi/kebert-base`로 지정했으므로 프리트레인을 마친 BERT로 `kcbert-base`를 사용한다.

모델을 초기화하는 코드에서 `BertForSequenceClassification`은 프리트레인을 마친 BERT 모델 위에 문서 분류용 태스크 모듈이 덧붙여진 형태의 모델 클래스이다. 이 클래스는 허깅페이스에서 제공하는 `transformers` 라이브러리에 포함되어 있다.

#### code 1-11

In [11]:
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)
model = BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

## 6. 모델 학습시키기

파이토치 라이트닝(pytorch lightning*)이 제공하는 `LightningModule` 클래스를 상속받아 태스크(task)를 정의한다. 태스크에는 모델과 옵티마이저, 학습 과정 등이 정의되어 있다.
* pytorch lightning*: github.com/PyTorchLightning/pytorch-lightning 

### Task 정의

code 1-3**(1. 각종 설정하기-모델 환경 설정)**에서 만든 학습 설정(`args`)과 code 1-11**(5. 모델 불러오기-모델초기화)**에서 준비한 모델(`model`)을 `ClassificationTask`에 주입한다. `ClassificationTask`에는 **옵티마이저**(optimizer), **러닝 레이트 스케줄러**(learnig rate scheduler)가 정의되어 있다. 옵티마이저로는 **아담**(Adam), 러닝 레이트 스케줄러로는 `ExponentialLR`을 사용한다.

code1-12를 통해 문서 분류용 태스크를 정의할 수 있다.

#### code 1-12

In [12]:
from ratsnlp.nlpbook.classification import ClassificationTask
task = ClassificationTask(model, args)

'Do it! 자연어 처리'의 저자 이기창 님의 비유를 인용하자면,

---
   *모델 학습 과정은 눈을 가린 상태에서 산등성이를 한 걸음씩 내려가는 과정과 같다. 러닝 레이트는 한 번 내려갈 때 얼마나 이동할지 보폭에 해당한다.*

---

학습이 진행되는 동안 점차 러닝 레이트를 줄여 세밀하게 탐색하면 좀 더 좋은 모델을 만들 수 있다. 이 역할을 하는 게 바로 러닝 레이트 스케줄러이다. `ExponentialLR`은 현재 에포크(epoch*)의 러닝 메이트를 '이전 에포크의 러닝 레이트 x gamma'로 스케줄링 한다. 본 예제에서는 gamma를 0.9로 설정하였다.
* 에포크: 데이터 전체를 학습하는 횟수. 만일 에포크가 3이라면 데이터를 3번 반복 학습한다는 뜻이다.


### 트레이너 정의

code 1-13을 통해 트레이너를 정의할 수 있다. 이 트레이너는 파이토치 라이트닝 라이브러리의 도움을 받아 GPU/TPU 설정, 로그 및 체크포인트 등 귀찮은 설정을 알아서 해 준다.

#### code 1-13

In [13]:
trainer = nlpbook.get_trainer(args)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


### 학습 개시

code 1-14처럼 트레이너의 `fit()`함수를 호출하면 학습을 시작한다. 학습 시간은 자신의 코랩 환경에 따라 다를 수 있으나 꽤 오래 걸릴 수 있다. 학습이 진행되는 도중 브라우저를 끄면 모델 학습을 비롯한 코랩 실행이 중단되니 주의해야한다.

#### code 1-14

In [14]:
trainer.fit(
    task,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
--------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
435.680   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]



---


# **문서 분류 모델 학습**을 요약하자면

# 각종 설정을 마친 뒤 말뭉치를 내려받아 전처리한다. 프리트레인을 마친 모델을 이 데이터에 맞게 파인 튜닝한다.
